In [36]:
from talus import morse
import scipy
import sklearn
import sklearn.datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [12]:
X, y = sklearn.datasets.load_iris(return_X_y=True)

In [14]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25, random_state=3)

In [15]:
train_X.shape

(112, 4)

In [24]:
batch_size=4

In [76]:
def build_morse_points(points, clf):
    morse_points = []
    predictions = clf.predict_proba(points)

    for i, point in enumerate(points):
        entropy = scipy.stats.entropy(predictions[i])
        morse_points.append(morse.MorseNode(identifier=i, value=entropy, vector=list(point)))
    return morse_points

In [77]:
def maximum_persistence_selection(available_points, clf, batch_size):
    morse_points = build_morse_points(available_points, clf)
    # FIXME: fit(X, values) is way cleaner than doing this all myself
    # FIXME: Rust panics on getting NaNs - should be an error but not a panic
    morse_result = talus.morse.persistence_by_knn(morse_points, k=batch_size)
    
    persistences = [(morse_result.descending_complex.lifetimes[i], i)
                    for i in range(len(available_points))]
    
    return [x[1] for x in sorted(persistences)[-batch_size:]]

In [88]:
selected_indices = list(np.random.choice(range(train_X.shape[0]), size=batch_size,
                                         replace=False))
remaining_indices = [i for i in range(len(train_X)) if i not in selected_indices]
index_to_point = dict(enumerate(train_X))
clf = RandomForestClassifier(n_estimators=50)
while len(remaining_indices) > 0:
    clf.fit(train_X[selected_indices], train_y[selected_indices])
    predictions = clf.predict(test_X)
    persistence_accuracy = sklearn.metrics.accuracy_score(test_y, predictions)
    
    print(f"Persistence accuracy: {persistence_accuracy}")

    
    next_points = maximum_persistence_selection([index_to_point[i] for i in remaining_indices],
                                                clf, batch_size)
    selected_indices.extend([remaining_indices[i] for i in next_points])
    remaining_indices = [i for i in range(len(train_X)) if i not in selected_indices]
    

Persistence accuracy: 0.5263157894736842
Persistence accuracy: 0.8157894736842105
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9736842105263158
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9736842105263158
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9736842105263158
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9736842105263158
Persistence accuracy: 0.9473684210526315
Persistence accuracy: 0.9736842105263158
Persistence accuracy: 0.9210526315789473
Persistence accuracy: 0.9736842105263158
Persistence accu